## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [52]:
import os
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import Error
from mysql.connector import ProgrammingError
from mysql.connector import InterfaceError

load_dotenv()

host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")

try:
    cnx =  mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )

    if cnx.is_connected():
        print("✅ Conexão estabelecida com sucesso!")
        # opcional: exibe versão do MySQL
        info = cnx.server_info
        print(f"Versão do servidor MySQL: {info}")


except ProgrammingError as pe:
    print("❌ Erro de autenticação ou comando inválido:")
    print(pe)

except InterfaceError as ie:
    print("❌ Erro de conexão (interface):")
    print(ie)

except Error as e:
    print("❌ Outro erro com MySQL:")
    print(e)

except Exception as ex:
    print("⚠️ Erro inesperado:")
    print(ex)



✅ Conexão estabelecida com sucesso!
Versão do servidor MySQL: 8.0.42-0ubuntu0.22.04.1


Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [ ]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [ ]:
cursor.execute('CREATE DATABASE IF NOT EXISTS dbprodutos;')

In [ ]:
cursor.execute('SHOW DATABASES;')

for db in cursor:
    print(db)

## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [ ]:
import pandas as pd

df_livros = pd.read_csv('/home/sander/repos/pipeline_dados_bd/data/tabela_livros.csv')
df_livros.head()

In [ ]:
leitor = df_livros.columns

for col in leitor:
    print(col, 'VARCHAR(100),')

In [ ]:
df_livros.shape

In [ ]:
cursor.execute("""
    create table if not exists dbprodutos.tb_livros(
            id VARCHAR(100),
            Produto VARCHAR(100),
            Categoria_Produto VARCHAR(100),
            Preço FLOAT(10,2),
            Frete FLOAT(10,2),
            Data_Compra DATE,
            Vendedor VARCHAR(100),
            Local_compra VARCHAR(100),
            Avaliação_compra INT,
            Tipo_pagamento VARCHAR(100),
            Qntd_parcelas INT,
            latitude FLOAT(10,2),
            longitude FLOAT(10,2),  

            primary key (id) 
               
    );

""")

**Selecionando a base de dados para verificar a tabela criada**

In [ ]:
cursor.execute('use dbprodutos;')
cursor.execute('show tables;')

for tb in cursor:
    print(tb)

In [ ]:
cursor.execute('truncate table tb_livros')

## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [ ]:
for i, row in df_livros.iterrows():
    print(tuple(row))

In [ ]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados

In [ ]:
sql = 'insert into dbprodutos.tb_livros values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'

cursor.executemany(sql, lista_dados)
cnx.commit()

In [ ]:
print(cursor.rowcount, 'dados inseridos')

## Desafio - Criar nova tabela tabela_2021_em_diante e inserir dados 

In [ ]:
df_tabela_2021 = pd.read_csv('/home/sander/repos/pipeline_dados_bd/data/tabela_2021_em_diante.csv')
df_livros.head()

In [ ]:
df_tabela_2021.shape

In [ ]:
organizador = df_tabela_2021.columns

for col in organizador:
    print(col, 'varchar(100)')


In [ ]:
df_tabela_2021.dtypes

In [ ]:
# Criar uma nova tabela chamada tb_produtos_2021_em_diante dentro da base de dados dbprodutos;

cursor.execute("""
    create table if not exists tb_produtos_2021_em_diante(
            id VARCHAR(100),
            Produto VARCHAR(100),
            Categoria_Produto VARCHAR(100),
            Preço FLOAT(10,2),
            Frete FLOAT(10,2),
            Data_Compra DATE,
            Vendedor VARCHAR(100),
            Local_compra VARCHAR(100),
            Avaliação_compra INT,
            Tipo_pagamento VARCHAR(100),
            Qntd_parcelas INT,
            latitude FLOAT(10,2),
            longitude FLOAT(10,2),  

            primary key (id) 
               
               );
""")

In [ ]:
cursor.execute('use dbprodutos;')
cursor.execute('show tables;')

for tb in cursor:
    print(tb)

In [ ]:
# Importar os dados contidos no arquivo tabela_2021_em_diante.csv e armazená-los na nova tabela criada.
cursor.execute('truncate table tb_produtos_2021_em_diante;')

In [ ]:
for i, row in df_tabela_2021.iterrows():
    print(tuple(row))

In [ ]:
lista_dados = [tuple(row) for i, row in df_tabela_2021.iterrows()]
lista_dados

In [ ]:
sql = 'insert into dbprodutos.tb_produtos_2021_em_diante values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'

cursor.executemany(sql, lista_dados)
cnx.commit()

In [ ]:
print(cursor.rowcount, 'dados inseridos')


In [ ]:
cursor.execute('select * from dbprodutos.tb_produtos_2021_em_diante;')

# Pega todos os resultados
resultados = cursor.fetchall()

# Exibe os resultados
for linha in resultados:
    print(linha)

## Visualizando os dados inseridos

In [ ]:
cursor.execute('select * from dbprodutos.tb_livros;')

colunas = [desc[0] for desc in cursor.description]
print(colunas)

for row in cursor:
    print(row)



In [47]:
cursor.close()

True

In [48]:
cnx.close()